In [1]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import random
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import threading
from sklearn.metrics import accuracy_score

In [2]:
# RANDOM FOREST CLASSIFER
def Random_forest(X_test, X_train, y_test, Y_train, i):
   rfc = RandomForestClassifier()
   rfc.fit(X_train,Y_train)
   y_pred = rfc.predict(X_test)
   accuracy = accuracy_score(y_test, y_pred)
   print(f"%d Accuracy from Random forest:  {accuracy}"  %(i))

In [3]:
#SUPPORT VECTOR MACHINE (SVM)
def support_vector(X_test, X_train, y_test, Y_train, i):
   clf = svm.SVC(kernel='linear')
   clf.fit(X_train, Y_train)
   y_pred = clf.predict(X_test)
   accuracy = accuracy_score(y_test, y_pred)
   print(f"%d Accuracy from SVM:  {accuracy}"  %(i))

In [4]:
#NAIVE BAYES
def Naive(X_test, X_train, y_test, Y_train, i):
   naive = GaussianNB()
   naive.fit(X_train, Y_train)
   y_pred = naive.predict(X_test)
   accuracy = accuracy_score(y_test, y_pred)
   print(f"%d Accuracy from Naive:  {accuracy}"  %(i))

In [5]:
#LOGISTIC REGRESSION
def LR(X_test, X_train, y_test, Y_train, i):
   logreg = LogisticRegression(max_iter=1000)
   logreg.fit(X_train, Y_train)
   y_pred = logreg.predict(X_test)
   accuracy = accuracy_score(y_test, y_pred)
   print(f"%d Accuracy from Logistic Regression:  {accuracy}"  %(i))

In [6]:
#DECISION TREE CLASSIFIER
def decision_tree(X_test, X_train, y_test, Y_train, i):
   clf = DecisionTreeClassifier()
   clf.fit(X_train, Y_train)
   y_pred = clf.predict(X_test)
   accuracy = accuracy_score(y_test, y_pred)
   print(f"%d Accuracy from Decision Tree:  {accuracy}"  %(i))

In [7]:
df = pd.read_csv("Creditcard_data.csv")
X = df.drop("Class", axis=1)
y = df["Class"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
sm = SMOTE(random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)
balanced_df = pd.concat([X_train_bal, y_train_bal], axis=1)

In [9]:
#Simple Random Sampling
n = int((1.96*1.96 * 0.5*0.5)/(0.05**2))
data_samp1 = balanced_df.sample(n, random_state=50)
X_train_samp1 = data_samp1.drop("Class", axis=1)
y_train_samp1 = data_samp1["Class"]

In [10]:
#Stratified sampling
X_train_samp2, _, y_train_samp2, _ = train_test_split(X_train_bal, y_train_bal,train_size=308, random_state=42, stratify=y_train_bal)

In [11]:
#Systematic Clustering
sample_size = 308
sampling_interval = len(X_train_bal) // sample_size
starting_point = random.randint(0, sampling_interval-1)
X_train_samp3 = pd.DataFrame()
y_train_samp3 = pd.Series(dtype=object)
for i in range(starting_point, len(X_train), sampling_interval):
   X_train_samp3 = pd.concat([X_train_samp3, X_train.iloc[[i]]])
   y_train_samp3 = pd.concat([y_train_samp3, y_train.iloc[[i]]])
   if len(X_train_samp3) == sample_size:
      break

In [12]:
#Cluster Sampling
kmeans = KMeans(n_clusters=10, random_state=42).fit(balanced_df)
cluster_assignments = kmeans.labels_
selected_clusters = [0, 2, 4, 5, 8]
cluster_series = pd.Series(cluster_assignments)
df_cluster_sample = balanced_df[cluster_series.isin(selected_clusters)]
X_train_samp4 = df_cluster_sample.drop("Class", axis=1)
y_train_samp4 = df_cluster_sample["Class"]

In [13]:
#Convenience sampling
convenience_sample = pd.concat([balanced_df.head(155),balanced_df.tail(155)])
X_train_samp5 = convenience_sample.drop("Class", axis=1)
y_train_samp5 = convenience_sample["Class"]

In [14]:
threads = []
for i in range (1,6):
   X_arg_name = 'X_train_samp{}'.format(i)
   X_arg_value = globals()[X_arg_name]
   y_arg_name = 'y_train_samp{}'.format(i)
   y_arg_value = globals()[y_arg_name]
   t1 = threading.Thread(target=Random_forest, args=(X_test, X_arg_value, y_test, y_arg_value, i,))
   t2 = threading.Thread(target=support_vector, args=(X_test, X_arg_value, y_test, y_arg_value, i,))
   t3 = threading.Thread(target=Naive, args=(X_test, X_arg_value, y_test, y_arg_value, i,))
   t4 = threading.Thread(target=LR, args=(X_test, X_arg_value, y_test, y_arg_value, i,))
   t5 = threading.Thread(target=decision_tree, args=(X_test, X_arg_value, y_test, y_arg_value, i,))
   t1.start()
   t2.start()
   t3.start()
   t4.start()
   t5.start()
   t1.join()
   t2.join()
   t3.join()
   t4.join()
   t5.join()

1 Accuracy from Decision Tree:  0.9525862068965517
1 Accuracy from Naive:  0.9396551724137931
1 Accuracy from Logistic Regression:  0.9094827586206896
1 Accuracy from Random forest:  0.9870689655172413
1 Accuracy from SVM:  0.9094827586206896
2 Accuracy from Naive:  0.9396551724137931
2 Accuracy from Decision Tree:  0.9396551724137931
2 Accuracy from Logistic Regression:  0.8879310344827587
2 Accuracy from Random forest:  0.9827586206896551
2 Accuracy from SVM:  0.8922413793103449
3 Accuracy from Naive:  0.9741379310344828
3 Accuracy from SVM:  0.9870689655172413
3 Accuracy from Decision Tree:  0.9741379310344828
3 Accuracy from Logistic Regression:  0.9870689655172413
3 Accuracy from Random forest:  0.9870689655172413
4 Accuracy from Naive:  0.9612068965517241
4 Accuracy from Decision Tree:  0.9612068965517241
4 Accuracy from Logistic Regression:  0.9051724137931034
4 Accuracy from Random forest:  0.9870689655172413
4 Accuracy from SVM:  0.9137931034482759
5 Accuracy from Naive:  0.93